## Demo for SDK

This example shows how to use SDK to deploy a task. The demo notebook includes the following steps:

- [Getting Started](#Getting-Started)
    - [initialization](#initialization)
    - [show available hardware info list](#show-available-hardware-information)
    - [choose hardware config](#Set-default-hardware-config-(Optional))
- [Creating And Deploying Task](#Creating-And-Deploying-Task)
    - [estimate payment amount](#estimate-payment-amount-optional)
    - [Auto Pay Path](#Auto-Pay-Path)
        - [create task with auto_pay](#Create-task-(auto_pay))
        - [renew task with auto-pay](#renew-task-auto-pay-optional)
    - [No Auto Pay Path](#No-Auto-Pay-Path)
        - [create task with no auto_pay to get `task_uuid`](#Create-task-(No-auto_pay))
        - [make payment](#make-payment-(Optional))
        - [validate payment](#validate-payment-to-deploy-task)
        - [renew task](#renew-task-(no-auto-pay)-optional)
    - [terminate task](#terminate-task-optional)
    - [claim review](#claim-review-optional)
- [View Deployed Task Results](#View-Deployed-Task-Results)
    - [follow up task status](#follow-up-task-status-optional)
    - [show result](#show-result)


# Getting Started


### Initialization

#### get an `API_KEY`

- For test version, get `API_KEY` in dashboard page: https://orchestrator-test.swanchain.io
- For prod version, get `API_KEY` in dashboard page: https://orchestrator.swanchain.io

If use this repository to test on your local machine, add `sys.path.insert(0, '..')` at the beginning, and run code in the root directory of this repository.

To use this SDK, you need to add environment file `.env` in your local directory, including the following parameters (`PK` is private key):

```
API_KEY=
WALLET=
PK=
```

In [22]:
import sys
sys.path.insert(0, '..') 

import os
import time
import dotenv
import json
dotenv.load_dotenv()

import swan

# dev_url = "https://swanhub-cali.swanchain.io"
# swan.setup_default_session(login_url=dev_url)
# session = swan._get_default_session()

In [23]:
wallet_address = os.getenv('WALLET')
private_key = os.getenv('PK')
api_key = os.getenv('API')
swan_orchestrator = swan.resource(api_key=api_key, service_name='Orchestrator')
# swan_orchestrator = swan.resource(api_key=api_key, service_name='Orchestrator', url_endpoint='dev_url')

In [24]:
r = swan_orchestrator.contract_info
print(json.dumps(r, indent=2))

{
  "client_contract_address": "0x9c5397F804f6663326151c81bBD82bb1451059E8",
  "payment_contract_address": "0xB48c5D1c025655BA79Ac4E10C0F19523dB97c816",
  "rpc_url": "https://rpc-atom-internal.swanchain.io",
  "swan_token_contract_address": "0x91B25A65b295F0405552A4bbB77879ab5e38166c"
}


### Show available hardware information

In [34]:
hardwares_info = swan_orchestrator.get_hardware_config()
hardwares_info
hardware_id = 1 # 'C1ae.medium'
region = 'North Carolina-US'

[{'id': 0,
  'name': 'C1ae.small',
  'description': 'CPU only · 2 vCPU · 2 GiB',
  'type': 'CPU',
  'region': ['Quebec-CA', 'North Carolina-US'],
  'price': '0.0',
  'status': 'available'},
 {'id': 1,
  'name': 'C1ae.medium',
  'description': 'CPU only · 4 vCPU · 4 GiB',
  'type': 'CPU',
  'region': ['Quebec-CA', 'North Carolina-US'],
  'price': '1.0',
  'status': 'available'},
 {'id': 12,
  'name': 'G1ae.small',
  'description': 'Nvidia 3080 · 4 vCPU · 8 GiB',
  'type': 'GPU',
  'region': ['Quebec-CA', 'North Carolina-US'],
  'price': '10.0',
  'status': 'available'},
 {'id': 13,
  'name': 'G1ae.medium',
  'description': 'Nvidia 3080 · 8 vCPU · 16 GiB',
  'type': 'GPU',
  'region': ['Quebec-CA', 'North Carolina-US'],
  'price': '11.0',
  'status': 'available'},
 {'id': 20,
  'name': 'Hpc1ae.small',
  'description': 'Nvidia 3090 · 4 vCPU · 8 GiB',
  'type': 'GPU',
  'region': ['Quebec-CA'],
  'price': '14.0',
  'status': 'available'},
 {'id': 21,
  'name': 'Hpc1ae.medium',
  'descripti

### Set default hardware config (Optional)

choose a default hardware with its hardware id and region. Set default `hardware_id` and `region` will be used in the steps to deploy task if not provided as parameters in future functions

In [28]:
hardware_id = 1 # 'C1ae.medium'
region = 'North Carolina-US'
if swan_orchestrator.set_default_task_config(hardware_id, region):
    print("Successfully set up default task configuration")

Successfully set up default task configuration


# Creating And Deploying Task

### Estimate Payment amount (Optional)

Estimate the cost of deploying the task on specified hardware for duration (in seconds)

In [29]:
duration = 1 * 3600 # 1 hour

amount = swan_orchestrator.estimate_payment(
    duration=duration, # Optional: Defaults to 3600 seconds or 1 hour
    hardware_id=hardware_id, # Optional: Defaults to hardware_id set in set_default_task_config or 0 (free) if not set
)

print(amount)

1


## Auto Pay Path

### Create task (auto_pay)

This step creates, pays and deploys task. It also gets `task_uuid`, which is useful in task info steps.

`wallet_address` is mandatory. `job_source_uri` is mandatory in this demo, please check out example-demo-prebuilt-image if deploying prebuilt images with `image_name`.

In this section, `private_key` is mandatory. If you do not want to use `private_key` and pay through SDK, please check out [create task with no auto_pay](#Create-task-(No-auto_pay))

`job_source_uri` is repository url of code to be deployed, must contain a must contain a dockerfile or deploy.yaml file

In [30]:
# job_source_uri = '<github repository url of code to be deployed>'

# Demo example: uncomment to use
job_source_uri = 'https://github.com/alphaflows/tetris-docker-image.git'

result = swan_orchestrator.create_task(
    wallet_address=wallet_address,
    job_source_uri=job_source_uri,
    auto_pay=True, # Optional: Defaults to false, but in this section's path, set to True
    private_key=private_key,
    hardware_id=hardware_id, # Optional: Defaults to hardware_id set in set_default_task_config or 0 (free) if not set
    region='North Carolina-US', # Optional: Defaults to region set in set_default_task_config or global if not set
    duration=duration, # Optional: Defaults to 3600 seconds
)

print(json.dumps(result, indent=2))

# Store the task_uuid of created task
task_uuid = result['id']

{'tx_hash': '0xc06adfb4d3bc7e31a641fd88bc8428c908d438eff8305ef6c4682ddd10802df3', 'task_uuid': 'a64452cf-fb1c-49bd-bfbe-bfe2b0b09f51'}
{
  "data": {
    "config_id": 2,
    "created_at": 1719593512,
    "duration": 3600,
    "ended_at": null,
    "error_code": null,
    "id": 2443,
    "order_type": "Creation",
    "refund_tx_hash": null,
    "region": "North Carolina-US",
    "space_id": null,
    "start_in": 300,
    "started_at": 1719593512,
    "status": "pending_payment_confirm",
    "task_uuid": "a64452cf-fb1c-49bd-bfbe-bfe2b0b09f51",
    "tx_hash": "0xc06adfb4d3bc7e31a641fd88bc8428c908d438eff8305ef6c4682ddd10802df3",
    "updated_at": 1719593520,
    "uuid": "9d94179f-fe0d-4079-a59c-0ca8b513d3d7"
  },
  "message": "Query order status success.",
  "status": "success",
  "tx_hash": "0xc06adfb4d3bc7e31a641fd88bc8428c908d438eff8305ef6c4682ddd10802df3",
  "id": "a64452cf-fb1c-49bd-bfbe-bfe2b0b09f51"
}


### renew task (auto-pay) (optional)

Extend `task_uuid` by `duration`. Using auto pay automatically makes a transaction to SWAN contract and extends the task.

In [32]:
renew_task = swan_orchestrator.renew_task(
    task_uuid=task_uuid, 
    duration=60, # Optional: Defaults to 3600 seconds (1 hour)
    auto_pay=True, # Optional: Defaults to False, in this demo path set to True
    private_key=private_key,
    hardware_id=hardware_id # Optional: Defaults to hardware_id set in set_default_task_config or 0 (free) if not set
)

if renew_task and renew_task['status'] == 'success':
    print(f"successfully renewed task")

successfully renewed task


If completed steps above, please go to [View Deployed Task Results](#View-Deployed-Task-Results) to view results, or [terminate task](#terminate-task) for more functions related to tasks

## No Auto Pay Path

### Create task (No auto_pay)

This creates task, but does not pay or deploy task. It gets `task_uuid`, which is useful in payment steps.

`wallet_address` is mandatory. `job_source_uri` is mandatory in this demo, please check out example-demo-prebuilt-image if deploying prebuilt images.

`job_source_uri` is repository url of code to be deployed, must contain a must contain a dockerfile or deploy.yaml file

In [8]:
# job_source_uri = '<github repository url of code to be deployed>'

# Demo example: uncomment to use
job_source_uri = 'https://github.com/alphaflows/tetris-docker-image.git'

result = swan_orchestrator.create_task(
    wallet_address=wallet_address,
    job_source_uri=job_source_uri,
    hardware_id=hardware_id, # Optional: Defaults to hardware_id set in set_default_task_config or 0 (free) if not set
    region='North Carolina-US', # Optional: Defaults to region set in set_default_task_config or global if not set
    duration=duration, # Optional: Defaults to 3600 seconds
    auto_pay=False, # Optional: Defaults to false
)

print(json.dumps(result, indent=2))

# Store the task_uuid of created task
task_uuid = result['id']

{
  "data": {
    "task": {
      "comments": null,
      "created_at": 1719592814,
      "end_at": 1719596414,
      "id": 4427,
      "leading_job_id": null,
      "refund_amount": null,
      "refund_wallet": "0x00165Ca5Ab193c4D2709F94A5418165bA04394E6",
      "source": "v2",
      "start_at": 1719592814,
      "start_in": 300,
      "status": "initialized",
      "task_detail": {
        "amount": 0.0,
        "bidder_limit": 3,
        "created_at": 1719592814,
        "duration": 3600,
        "end_at": 1719596414,
        "hardware": "C1ae.small",
        "job_result_uri": null,
        "job_source_uri": "https://plutotest.acl.swanipfs.com/ipfs/QmcgTP2tXsxGAG3eBsa5fTPiAxgMPRPW4cKUwLy2PMGWu6",
        "price_per_hour": "0.0",
        "requirements": {
          "hardware": "None",
          "hardware_type": "CPU",
          "memory": "2",
          "region": "North Carolina-US",
          "vcpu": "2"
        },
        "start_at": 1719592814,
        "status": "paid",
        "st

### Make Payment (Optional)

This step is using `task_uuid`, `private_key`,  `duration`, and `hardware_id` to submit payment and make task eligible for assigning if payment successful via swan SDK. `task_uuid`, `private_key` are mandatory.

If following this section, please skip to - [View Deployed Task Results](#View-Deployed-Task-Results) next.
If do not want to submit payment with swan SDK, and directly made payment to swan contract instead, please move onto next section.

In [11]:
if result_validation := swan_orchestrator.make_payment(
    task_uuid=task_uuid, 
    private_key=private_key,
    duration=duration, # Optional: Defaults to 3600 seconds (1 hour)
    hardware_id=hardware_id # Optional: Defaults to hardware_id set in set_default_task_config or 0 (free) if not set
):
    print(json.dumps(result_validation, indent=2))
else:
    print('validation failed')

### Validate Payment to deploy task

Only use this if paid directly to contract. If used make_payment section above, ignore this section.

This step will use `tx_hash` and `task_uuid` to validate the payment and then make task eligible for assigning if validation successful

Will Delete the submit_payment code block below in final version

In [10]:
tx_hash = swan_orchestrator.submit_payment(
    task_uuid=task_uuid, 
    private_key=private_key
)

In [12]:
if result_validation := swan_orchestrator.validate_payment(
    tx_hash=tx_hash, # "<tx_hash of payment of task_uuid to swan contract>",
    task_uuid=task_uuid
):
    print(json.dumps(result_validation, indent=2))
else:
    print('validation failed')

{'tx_hash': '0x2f00af7c5e936cab22f1b51a1329fd6d86cd04c9aba62c494d2035408508cfcc', 'task_uuid': 'b30c11fe-0714-4f46-86ae-493d23ce269f'}
{
  "data": {
    "config_id": 1,
    "created_at": 1719592814,
    "duration": 3600,
    "ended_at": null,
    "error_code": null,
    "id": 2441,
    "order_type": "Creation",
    "refund_tx_hash": null,
    "region": "North Carolina-US",
    "space_id": null,
    "start_in": 300,
    "started_at": 1719592814,
    "status": "pending_payment_confirm",
    "task_uuid": "b30c11fe-0714-4f46-86ae-493d23ce269f",
    "tx_hash": "0x2f00af7c5e936cab22f1b51a1329fd6d86cd04c9aba62c494d2035408508cfcc",
    "updated_at": 1719592940,
    "uuid": "827848b5-5a6f-46c7-837d-03d780c4e425"
  },
  "message": "Query order status success.",
  "status": "success"
}


### Renew Task (no auto-pay) (Optional)

Extend `task_uuid` by `duration`. `tx_hash` of payment for extension required.

Will Delete the submit_payment code block below in final version

In [13]:
tx_hash = swan_orchestrator.submit_payment(
    task_uuid=task_uuid, 
    private_key=private_key,
    duration=60, # Optional: Defaults to 3600 seconds (1 hour)
    hardware_id=hardware_id # Optional: Defaults to hardware_id set in set_default_task_config or 0 (free) if not set
)

In [15]:
renew_task = swan_orchestrator.renew_task(
    task_uuid=task_uuid, 
    duration=60, # Optional: Defaults to 3600 seconds (1 hour)
    tx_hash=tx_hash, # tx_hash of payment to swan contract for this task
    hardware_id=hardware_id # Optional: Defaults to hardware_id set in set_default_task_config or 0 (free) if not set
)

if renew_task and renew_task['status'] == 'success':
    print(f"successfully renewed {task_uuid}")
else:
    print(f"Unable to renew {task_uuid}")

successfully renewed task


### terminate task (Optional)

Terminate the task `task_uuid` and get a refund for remaining time

In [36]:
terminate_status = swan_orchestrator.terminate_task(task_uuid)
if terminate_status['status'] == 'success':
    print(f"Terminated {task_uuid} successfully")
else:
    print(f"Failed to terminate {task_uuid}")

Terminated a64452cf-fb1c-49bd-bfbe-bfe2b0b09f51 successfully


### claim review (Optional)

Review if `task_uui` uptime is above 90% and give refund if below 90%

In [ ]:
claim_review = swan_orchestrator.claim_review(task_uuid)
print(claim_review)

## View Deployed Task Results

### follow up task status (optional)
The following step is optional, shows information when waiting for task being deployed.

In [16]:
# Check task info
info = swan_orchestrator.get_deployment_info(task_uuid=task_uuid)
print(json.dumps(info, indent=2))

{
  "data": {
    "computing_providers": [
      {
        "beneficiary": "0x1AE8665873565C9089753920A0866C49Bd35DCC9",
        "cp_account_address": "0xd63853b23Af4cc3b37377368dd33F09b8567aCea",
        "created_at": 1718386123,
        "id": 71,
        "lat": 35.8639,
        "lon": -78.535,
        "multi_address": [
          "/ip4/provider.cp.filezoo.com.cn/tcp/9087"
        ],
        "node_id": "0475e6cb8909e125db8b233e9d2c8237e28e8d852fb4dcf1d5f9b5eead94931d8ed84c5f08e4e88a53b24d80bb3bb3b181389fcddf0114ac93f2866d769cc8ff18",
        "online": 1,
        "owner_address": "0x1AE8665873565C9089753920A0866C49Bd35DCC9",
        "region": "North Carolina-US",
        "task_types": "[3]",
        "updated_at": 1719593022,
        "version": "2.0",
        "worker_address": "0x1AE8665873565C9089753920A0866C49Bd35DCC9"
      }
    ],
    "jobs": [
      {
        "build_log": "wss://log.cp.filezoo.com.cn:9087/api/v1/computing/lagrange/spaces/log?space_id=QmcgTP2tXsxGAG3eBsa5fTPiAxgMPRP

### Show result

`job_real_uri` is for show the result of application you deployed.  
You can put it into the web browser to view application.

In [31]:
result_url = swan_orchestrator.get_real_url(task_uuid)
print(result_url)

['https://xj9e7qr1eu.cp.filezoo.com.cn']


Check the response codes of the result_url

In [19]:
import requests
import json

headers = {
    'Content-Type': 'application/json',
}

for url in result_url:
    response = requests.get(url, headers=headers)

    try:
        print(json.dumps(response.json(), indent=4))
    except Exception as e:
        print(e)
        print(response)


Expecting value: line 1 column 1 (char 0)
<Response [200]>
